In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import gurobipy

In [2]:
Elec_Demand = pd.read_csv('Electricity_Demand.csv')
Hydro_Demand = pd.read_csv('Hydrogen_Demand.csv')

Elec_Price = pd.read_csv('Electricity_Prices.csv')
Solar_Supply = pd.read_csv('Solar_Forecast.csv')

In [3]:
# display(Elec_Demand)
# display(Hydro_Demand)
# display(Elec_Price)
# display(Solar_Supply)

In [4]:
# Manipulate data to get it into a convienient form
Solar_Supply['Datetime'] = pd.to_datetime(Solar_Supply['Datetime'])
Solar_Supply['Hour'] = Solar_Supply['Datetime'].dt.hour
cols = ['Datetime', 'Hour', '% of Rated Output']
Solar_Supply = Solar_Supply[cols]
Solar_Supply['MWh'] = Solar_Supply['% of Rated Output']*2

In [5]:
# Define Parameters
sE = Solar_Supply['MWh'].to_numpy() # Number of MWs of electricity produced at solar farm for hour i
pE = Elec_Price['Price ($/MWh)'].to_numpy() # Electricity price per MWh at hour i
pH = np.array([10 for i in range(pE.size)]) # Price of hydrogen per kg at hour i
dE = Elec_Demand['Demand (MWh)'].to_numpy() # Electricity demand in MWh at hour i
dH = Hydro_Demand['Demand (kg)'].to_numpy() # Hydrogen demand in kg at hour i

# Define Decision Variables
xE = cp.Variable(pE.size) # Number of MWhs of electricity purchased at hour i
xH = cp.Variable(pE.size) # Number of kgs of hydrogen purchased at hour i
nH = cp.Variable(pE.size) # Number of kgs of hydrogen in inventory at end of hour i

constraints = [xE[0] + sE[0] >= dE[0],                      # Electricity Demand Constraint at hour 1
               xH[0] >= dH[0] + nH[0],                      # Hydrogen Demand Constraint at hour 1
               xE[0] >= 0, xH[0] >= 0, nH[0] >=0]           # Sign constraints at hour 1
for i in range(1,pE.size):
    constraints += [xE[i] + sE[i] >= dE[i],                 # Electricity Demand Constraint at hour 2 to i
                    nH[i-1] + xH[i] >= dH[i] + nH[i],       # Hydrogen Demand Constraint at hour 2 to i
                    xE[i] >= 0, xH[i] >= 0, nH[i] >=0]      # Sign constraints at hour 2 to i

objective = cp.Minimize(pE.T@xE + pH.T@xH)                  # Minimize the cost

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.GUROBI)
print("The optimal value is:", problem.value)
print("The first 5 dual values solutions are:")
print("Electricity Demand Constraint:", problem.constraints[0].dual_value) # Dual value for the Electricity Demand Constraint at hour 1
print("Hydrogen Demand Constraint:", problem.constraints[1].dual_value) # Dual value for the Hydrogen Demand Constraint at hour 1
print("xE Sign Constraint:", problem.constraints[2].dual_value) # Dual value for the xE Sign Constraint at hour 1
print("xH Sign Constraint:", problem.constraints[3].dual_value) # Dual value for the xH Sign Constraint at hour 1
print("nH Sign Constraint:", problem.constraints[4].dual_value) # Dual value for the nH Sign Constraint at hour 1

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-03
The optimal value is: 17916.012987517464
The first 5 dual values solutions are:
Electricity Demand Constraint: 22.87
Hydrogen Demand Constraint: 10.0
xE Sign Constraint: -0.0
xH Sign Constraint: -0.0
nH Sign Constraint: -0.0


In [13]:
Solution_DataFrame = pd.DataFrame()
Solution_DataFrame['Energy Needed'] = dE
Solution_DataFrame['Solar Energy Generated'] = sE
Solution_DataFrame['Energy Purchased'] = xE.value
Solution_DataFrame['Energy Cost at Hour'] = pE
Solution_DataFrame['Hydrogen Needed'] = dH
Solution_DataFrame['Hydrogen Stored'] = nH.value
Solution_DataFrame['Hydrogen Purchased'] = xH.value
Solution_DataFrame['Hydrogen Cost at Hour'] = pH
Solution_DataFrame['Total Cost at Hour'] = (Solution_DataFrame['Energy Purchased'].values * Solution_DataFrame['Energy Cost at Hour'].values
                                            + Solution_DataFrame['Hydrogen Purchased'].values * Solution_DataFrame['Hydrogen Cost at Hour'].values)
Solution_DataFrame['Total Cost'] = Solution_DataFrame['Total Cost at Hour'].cumsum(axis=0)
Solution_DataFrame.index.name = 'Hour'
Solution_DataFrame.index = Solution_DataFrame.index+1
Solution_DataFrame.to_csv('output.csv')
# display(Solution_DataFrame)

In [14]:
display(Solution_DataFrame)

,Energy Needed,Solar Energy Generated,Energy Purchased,Energy Cost at Hour,Hydrogen Needed,Hydrogen Stored,Hydrogen Purchased,Hydrogen Cost at Hour,Total Cost at Hour,Total Cost
Hour,,,,,,,,,,
1,0.224875,0.0,0.224875,22.87,32.303293,1700.310550,1732.613842,10,17331.281325,17331.281325
2,0.267280,0.0,0.267280,7.09,10.136755,1690.173795,0.000000,10,1.895012,17333.176337
3,0.825336,0.0,0.825336,7.09,27.006698,1663.167097,0.000000,10,5.851634,17339.027971
4,0.500773,0.0,0.500773,7.09,39.842841,1623.324256,0.000000,10,3.550480,17342.578451
5,0.275956,0.0,0.275956,0.00,3.644255,1619.680001,0.000000,10,0.000000,17342.578451
...,...,...,...,...,...,...,...,...,...,...
68,0.591405,0.0,0.591405,11.58,23.550953,36.731405,0.000000,10,6.848474,17738.782355
69,0.530659,0.0,0.530659,11.19,7.903277,28.828128,0.000000,10,5.938071,17744.720426
70,0.255750,0.0,0.255750,7.32,22.241644,6.586485,0.000000,10,1.872090,17746.592516


In [15]:
problem.variables

<bound method Problem.variables of Problem(Minimize(Expression(AFFINE, UNKNOWN, ())), [Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Expression(AFFINE, UNKNOWN, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Expression(AFFINE, UNKNOWN, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Expression(AFFINE, UNKNOWN, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, NONNEGATIVE, ())), Inequality(Expression(AFFINE, UNKNOWN, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, ZERO, ())), Inequality(Constant(CONSTANT, NON

In [11]:
np.sum( xE.value )

18.008888533060002

In [12]:
np.sum(xH.value)

1749.3604153040003

In [17]:
tab = pd.read_csv("output.csv")
Solution_DataFrame.head()

,Energy Needed,Solar Energy Generated,Energy Purchased,Energy Cost at Hour,Hydrogen Needed,Hydrogen Stored,Hydrogen Purchased,Hydrogen Cost at Hour,Total Cost at Hour,Total Cost
Hour,,,,,,,,,,
1,0.224875,0.0,0.224875,22.87,32.303293,1700.310550,1732.613842,10,17331.281325,17331.281325
2,0.267280,0.0,0.267280,7.09,10.136755,1690.173795,0.000000,10,1.895012,17333.176337
3,0.825336,0.0,0.825336,7.09,27.006698,1663.167097,0.000000,10,5.851634,17339.027971
4,0.500773,0.0,0.500773,7.09,39.842841,1623.324256,0.000000,10,3.550480,17342.578451
5,0.275956,0.0,0.275956,0.00,3.644255,1619.680001,0.000000,10,0.000000,17342.578451


In [19]:
tab.head()

,Hour,Energy Needed,Solar Energy Generated,Energy Purchased,Energy Cost at Hour,Hydrogen Needed,Hydrogen Stored,Hydrogen Purchased,Hydrogen Cost at Hour,Total Cost at Hour,Total Cost
0,1,0.224875,0.0,0.224875,22.87,32.303293,1700.310550,1732.613842,10,17331.281325,17331.281325
1,2,0.267280,0.0,0.267280,7.09,10.136755,1690.173795,0.000000,10,1.895012,17333.176337
2,3,0.825336,0.0,0.825336,7.09,27.006698,1663.167097,0.000000,10,5.851634,17339.027971
3,4,0.500773,0.0,0.500773,7.09,39.842841,1623.324256,0.000000,10,3.550480,17342.578451
4,5,0.275956,0.0,0.275956,0.00,3.644255,1619.680001,0.000000,10,0.000000,17342.578451
